In [3]:
import gradio as gr
import requests

def upload_and_analyze(file_obj, instruction_text):
    try:
        with open(file_obj.name, "rb") as f:
            files = {'file': ("uploaded.csv", f, "text/csv")}
            data = {'instruction_text': instruction_text}
            response = requests.post("http://localhost:8080/analyze_vibration/", files=files, data=data)

        if response.status_code == 200:
            result = response.json()
            # 這裡對應 FastAPI 回傳內容
            instructions = result.get("instructions", [])
            results = result.get("results", [])
            summary = result.get("llm_summary", "")

            if not results:
                summary = "⚠️ 沒有成功解析或計算特徵，請檢查指令與資料格式是否正確。"

            return instructions, results, summary
        else:
            return {}, {}, f"❌ 錯誤狀態碼：{response.status_code}"
    except Exception as e:
        return {}, {}, f"⚠️ 例外錯誤：{str(e)}"

gr.Interface(
    fn=upload_and_analyze,
    inputs=[
        gr.File(label="📤 上傳振動 CSV", file_types=[".csv"]),
        gr.Textbox(label="🧠 自然語言分析指令", placeholder="例：請對 Z 軸做 1500~4000Hz 濾波並計算 RMS 與轉速", lines=2)
    ],
    outputs=[
        gr.JSON(label="📋 指令解析結果（LLM 回傳）"),
        gr.JSON(label="📊 特徵值分析結果（多軸/多濾波）"),
        gr.Textbox(label="🧠 LLM 自然語言診斷摘要", lines=12)
    ],
    title="📈 振動訊號 AI 智能分析系統",
    description="請上傳包含 X, Y, Z 欄位的振動 CSV 檔案，並用自然語言輸入分析需求，系統會整合 MCP 與 LLM 自動處理。"
).launch(
    server_name="0.0.0.0",
    server_port=7871,
    share=False
)

* Running on local URL:  http://0.0.0.0:7871
* To create a public link, set `share=True` in `launch()`.
